# Reforestum
https://reforestum.com

In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
with open('../input/Reforestum/forests.json', 'r') as file:
    data = json.load(file)
type(data)

In [ ]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
df.info()

In [ ]:
df['country'] = df['data.forests.location_desc'].astype(str).apply(lambda x: x.split()[-1] if isinstance(x, str) else '')
df.head()

In [ ]:
columns_rename_mapping = {
    'data.forests.id': 'project_id_reported',
    'data.forests.coordinates': 'geometry',
    'data.forests.short_desc': 'project_description_reported',
    'country': 'country',
    'data.forests.reforested_surface':'site_sqkm'
}
df = df[list(columns_rename_mapping.keys())]
df.rename(columns=columns_rename_mapping, inplace=True)

### Fix geometries

In [ ]:
df['geometry'] = df['geometry'].apply(lambda x: Point(x))

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

### Harmonize nomenclature

In [ ]:
gdf['site_sqkm'] = gdf['site_sqkm'] / 1e6
gdf['url'] = 'https://reforestum.com'
gdf['host_name'] = 'reforestum'

In [ ]:
gdf = gdf.assign(site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,country=None,site_id_reported=None)
gdf.info()

In [ ]:
gdf.to_file('../midsave/reforestum.gpkg')